# Clustering of Toronto Neighbourhoods

In [1]:
import pandas as pd
import numpy as np

## Part1: Data scraping 

In [2]:
# Scrape the data with BeautifulSoup

# import the library we use to open URLs
import urllib.request
# import the BeautifulSoup library so we can parse HTML and XML documents
from bs4 import BeautifulSoup


# 1. specify which URL/web page we are going to be scraping
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
# 2. open the url using urllib.request and put the HTML into the page variable
page = urllib.request.urlopen(url)
# 3. parse the HTML from our URL into the BeautifulSoup parse tree format
soup = BeautifulSoup(page, "html.parser")

# 4. use the 'find_all' function to bring back all instances of the 'table' tag in the HTML and store in 'all_tables' variable
all_tables=soup.find_all("table")
# 5. use the 'find function' to bring back the wiki sort table
right_table=soup.find('table', class_='wikitable sortable')

# 6. identify the row entries of each collumn
ColPC=[]
ColBo=[]
ColNH=[]


for row in right_table.findAll('tr'):
    cells=row.findAll('td')
    if len(cells)==3:
        ColPC.append(cells[0].find(text=True))
        ColBo.append(cells[1].find(text=True))
        ColNH.append(cells[2].find(text=True))

        
# 7. build the dataframe from the scaped data
dfTor=pd.DataFrame(ColPC,columns=['PostalCode'])
dfTor['Borough']=ColBo
dfTor['Neighborhood']=ColNH


In [3]:
# Data wrangeling 
# 1. drop rows without assigned borough
dfTor=dfTor[dfTor.Borough != 'Not assigned\n']

# 2. update the index
dfTor.reset_index(drop=True,inplace=True)


# 3. remove all the white space as it will most likely cause trouble later
dfTor = dfTor.replace('\n','', regex=True)
# 4. test that there is no Borough without Neighborhood
dfTorTest=dfTor[dfTor.Neighborhood == 'Not assigned']
print('Length of test set:' + str(dfTorTest.shape[0]) + ' row index 0? - you are good; row index >0? replace the neighborhood by borough')

# 5. display the final data set
dfTor.head(12)



Length of test set:0 row index 0? - you are good; row index >0? replace the neighborhood by borough


,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


In [4]:
# length of final Data Frame
print ('The data set contains of: '+ str(dfTor.shape[0]) + ' rows')

The data set contains of: 103 rows


## Part 2:  Longitude and Lattitude of Post Codes

In [5]:
#import geospacial data as csv file as the geocoder did not return values within reasonable time
df_DatLaLo = pd.read_csv('https://cocl.us/Geospatial_data/Geospatial_Coordinates.csv')
print('Data downloaded and read into a dataframe!')

Data downloaded and read into a dataframe!


In [6]:
#loop over the toronto postal codes to assign a latitude and longitude to each neighborhood
# 1. ialize the columnds
ColLa=[]
ColLo=[]

# 2. for each index in the toronto neighborhood data frame
for ind in dfTor.index: 
    # 3. extract the post code
    PC=dfTor['PostalCode'][ind]
    # 4. find the post code in the list of latitude and longitude
    idx = df_DatLaLo[df_DatLaLo['Postal Code']==PC].index.values
    # 5. save the data of latitude and longitude in column
    ColLa.append(float(df_DatLaLo['Latitude'][idx].values))
    ColLo.append(float(df_DatLaLo['Longitude'][idx].values))
        
# 6. construct the new dataframe with latitude and longitude
dfTorLaLo=dfTor
dfTorLaLo['Latitude']=ColLa
dfTorLaLo['Longitude']=ColLo
dfTorLaLo.head(12)


,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",43.667856,-79.532242
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
7,M3B,North York,Don Mills,43.745906,-79.352188
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937


## Part 3: Clustering neighborhoods in Toronto 

In [7]:
# reduce the dataframe to only borough with the substring Toronto 

dfTorLaLo = dfTorLaLo[dfTorLaLo['Borough'].str.contains("Toronto")]


In [8]:
import json # library to handle JSON files


import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't installed it yet
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


Creating the map of Toronto, centered at downtown Toronto with the markers of the neighborhood 

In [9]:
# create map of Toronto using latitude and longitude values of Downtown Toronto
latitude=43.654260
longitude=-79.360636
map_Toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(dfTorLaLo['Latitude'], dfTorLaLo['Longitude'], dfTorLaLo['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Toronto)  
    
map_Toronto

Use Folium to get the data of venues at each neighborhood

In [10]:
# The code was removed by Watson Studio for sharing.

Define the getNearbyVenues function to retrieve data for neighborhoods

In [11]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Run the Function over all the neighborhoods and retreive the data (Top 100 venues)

In [12]:
LIMIT=100
Toronto_venues = getNearbyVenues(names=dfTorLaLo['Neighborhood'],
                                  latitudes=dfTorLaLo['Latitude'],
                                  longitudes=dfTorLaLo['Longitude']
                                 )

Regent Park, Harbourfront
Queen's Park, Ontario Provincial Government
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
The Danforth West, Riverdale
Toronto Dominion Centre, Design Exchange
Brockton, Parkdale Village, Exhibition Place
India Bazaar, The Beaches West
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West, Forest Hill Road Park
High Park, The Junction South
North Toronto West,  Lawrence Park
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
University of Toronto, Harbord
Runnymede, Swansea
Moore Park, Summerhill East
Kensington Market, Chinatown, Grange Park
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport


Check the dataframe of venues in Toronto

In [13]:
print(Toronto_venues.shape)
Toronto_venues.head()

(1627, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Regent Park, Harbourfront",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Regent Park, Harbourfront",43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
3,"Regent Park, Harbourfront",43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
4,"Regent Park, Harbourfront",43.65426,-79.360636,Corktown Common,43.655618,-79.356211,Park


Check the number of venues per neighborhood, just to be sure that there is enough going on in these places...

In [14]:
Toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,58,58,58,58,58,58
"Brockton, Parkdale Village, Exhibition Place",24,24,24,24,24,24
"Business reply mail Processing Centre, South Central Letter Processing Plant Toronto",19,19,19,19,19,19
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",16,16,16,16,16,16
Central Bay Street,67,67,67,67,67,67
Christie,17,17,17,17,17,17
Church and Wellesley,80,80,80,80,80,80
"Commerce Court, Victoria Hotel",100,100,100,100,100,100
Davisville,35,35,35,35,35,35


Drop all the neighborhoods that have less than 20 venues,
because they are mostlikely "unique" when using the clustering algorithm 
and they are also unattractive when considering to move to a new neighborhood


In [15]:
minVenue=20
VenCount=Toronto_venues.groupby('Neighborhood').count()>=minVenue
VenCount=VenCount[VenCount['Venue']]
# reduce the toronto venues list
Toronto_venues=Toronto_venues[Toronto_venues['Neighborhood'].isin(VenCount.index)]
# reduce the Toronto data set
dfTorLaLo=dfTorLaLo[dfTorLaLo['Neighborhood'].isin(VenCount.index)]


Analysing each neighborhood to see the cathegories of venues in each neighborhood

In [16]:
# one hot encoding and grouping of venues
Toronto_onehot = pd.get_dummies(Toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Toronto_onehot['Neighborhood'] = Toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [Toronto_onehot.columns[-1]] + list(Toronto_onehot.columns[:-1])
Toronto_onehot = Toronto_onehot[fixed_columns]

Toronto_onehot.head()

Toronto_grouped = Toronto_onehot.groupby('Neighborhood').mean().reset_index()
Toronto_grouped

,Neighborhood,Yoga Studio,Afghan Restaurant,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,...,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store
0,Berczy Park,0.000000,0.0000,0.000000,0.000000,0.00,0.017241,0.000000,0.000000,0.000000,...,0.0000,0.000000,0.000000,0.00,0.017241,0.000000,0.000000,0.000000,0.0000,0.000000
1,"Brockton, Parkdale Village, Exhibition Place",0.000000,0.0000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,...,0.0000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000
2,Central Bay Street,0.014925,0.0000,0.000000,0.000000,0.00,0.000000,0.014925,0.000000,0.000000,...,0.0000,0.000000,0.000000,0.00,0.014925,0.000000,0.000000,0.014925,0.0000,0.000000
3,Church and Wellesley,0.025000,0.0125,0.012500,0.000000,0.00,0.000000,0.000000,0.012500,0.000000,...,0.0125,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.0125,0.000000
4,"Commerce Court, Victoria Hotel",0.000000,0.0000,0.040000,0.000000,0.00,0.010000,0.000000,0.000000,0.010000,...,0.0000,0.000000,0.000000,0.00,0.020000,0.000000,0.000000,0.010000,0.0000,0.000000
5,Davisville,0.000000,0.0000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,...,0.0000,0.028571,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000
6,"First Canadian Place, Underground city",0.000000,0.0000,0.030000,0.000000,0.00,0.010000,0.000000,0.000000,0.030000,...,0.0000,0.000000,0.000000,0.01,0.010000,0.000000,0.000000,0.010000,0.0000,0.000000
7,"Garden District, Ryerson",0.000000,0.0000,0.000000,0.000000,0.00,0.010000,0.000000,0.000000,0.000000,...,0.0000,0.000000,0.000000,0.00,0.000000,0.010000,0.010000,0.010000,0.0000,0.000000
8,"Harbourfront East, Union Station, Toronto Islands",0.000000,0.0000,0.000000,0.000000,0.05,0.010000,0.000000,0.000000,0.000000,...,0.0000,0.000000,0.000000,0.01,0.010000,0.000000,0.000000,0.010000,0.0000,0.000000
9,"High Park, The Junction South",0.000000,0.0000,0.000000,0.043478,0.00,0.000000,0.000000,0.043478,0.000000,...,0.0000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000


create a pandas dataframe that lists the top10 venues for each neighborhood

Function to sor the venues in descending order

In [17]:
# function to sort venues in descending order
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]


creating the data frame

In [18]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = Toronto_grouped['Neighborhood']

for ind in np.arange(Toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Cocktail Bar,Cheese Shop,Beer Bar,Seafood Restaurant,Bakery,Restaurant,Café,Pub,Eastern European Restaurant
1,"Brockton, Parkdale Village, Exhibition Place",Café,Performing Arts Venue,Coffee Shop,Breakfast Spot,Grocery Store,Furniture / Home Store,Nightclub,Convenience Store,Pet Store,Restaurant
2,Central Bay Street,Coffee Shop,Italian Restaurant,Japanese Restaurant,Sandwich Place,Café,Middle Eastern Restaurant,Bubble Tea Shop,Burger Joint,Department Store,Dessert Shop
3,Church and Wellesley,Coffee Shop,Sushi Restaurant,Japanese Restaurant,Gay Bar,Restaurant,Yoga Studio,Men's Store,Pub,Hotel,Café
4,"Commerce Court, Victoria Hotel",Coffee Shop,Restaurant,Café,Hotel,American Restaurant,Gym,Japanese Restaurant,Italian Restaurant,Seafood Restaurant,Deli / Bodega


Clustering the neighborhoods

In [19]:
# set number of clusters
kclusters = 5

Toronto_grouped_clustering = Toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

Toronto_merged = dfTorLaLo

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
Toronto_merged = Toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

Toronto_merged.head() # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,2,Coffee Shop,Pub,Bakery,Park,Breakfast Spot,Café,Theater,Health Food Store,Event Space,Performing Arts Venue
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,2,Coffee Shop,Sushi Restaurant,Yoga Studio,Beer Bar,Burrito Place,Café,Sandwich Place,College Auditorium,Creperie,Diner
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,2,Clothing Store,Coffee Shop,Italian Restaurant,Café,Middle Eastern Restaurant,Bubble Tea Shop,Japanese Restaurant,Cosmetics Shop,Pizza Place,Fast Food Restaurant
15,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,2,Coffee Shop,Café,Cocktail Bar,Gastropub,American Restaurant,Gym,Clothing Store,Italian Restaurant,Restaurant,Hotel
20,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,2,Coffee Shop,Cocktail Bar,Cheese Shop,Beer Bar,Seafood Restaurant,Bakery,Restaurant,Café,Pub,Eastern European Restaurant


visualize the Toronto clusters of neighborhoods

In [20]:
# create map
# create map of Toronto using latitude and longitude values of Downtown Toronto
latitude=43.654260
longitude=-79.360636
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Toronto_merged['Latitude'], Toronto_merged['Longitude'], Toronto_merged['Neighborhood'], Toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters